In [16]:
import numpy as np 
import pandas as pd 
import os


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tqdm import tqdm
tqdm.pandas()

import lightgbm as lgb
import xgboost as xgb
from xgboost.sklearn import  XGBClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, log_loss, f1_score, accuracy_score, make_scorer

from keras.models import Sequential
from keras.layers import Dense, Input, Activation
from keras import optimizers, callbacks 


from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization,Add,Dropout
from keras.models import Model, load_model

from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils, plot_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
import collections

import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_validate

In [2]:
train = pd.read_csv('stats/train_final.csv')
test = pd.read_csv('stats/test_final.csv')
sample_sub = pd.read_csv('../data/sample_submission.csv')
print('train shape {}'.format(train.shape))
print('test shape {}'.format(test.shape))

train shape (2494, 6094)
test shape (1074, 6093)


In [4]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train['Crop_Id_Ne']),
                                                 train['Crop_Id_Ne'])
class_weights = list(class_weights)

# y_train= pd.get_dummies(train['Crop_Id_Ne'])
y_train = train['Crop_Id_Ne']

X_train = train.drop(['Field_Id','Crop_Id_Ne', 'Subregion'], axis = 1)
X_train.reset_index(inplace = True, drop= True)
X_test = test.drop(['Field_Id', 'Subregion'], axis = 1)
X_test.reset_index(inplace = True, drop = True)

## use only the bands mean values

In [6]:
dates = ["2017-01-01", "2017-01-31", "2017-02-10", "2017-03-12", "2017-03-22", "2017-05-31", "2017-06-20", 
        "2017-07-10", "2017-07-15", "2017-08-04", "2017-08-19"]

features = ['red', 'green', 'blue', 'band5', 'band6', 'band7', 'nir', 'band8a', 'band11', 'band12',
           'ndvi', 'reip', 'datt3', 'gemi', 'msbi', 'ccci', 'avi', 'cvi', 'ndsi', 'siwsi', 'savi', 'evi', 'ndre']

mean_columns = []
for date in dates:
    for layer in features:
        mean_columns.append(layer + '_mean'+date)
print(len(mean_columns))

253


In [12]:
X_train = train[mean_columns]
X_test = test[mean_columns]
y_train = train['Crop_Id_Ne']

In [17]:
i=1
kf = StratifiedKFold(n_splits=3,random_state=1,shuffle=True)
for train_index,test_index in kf.split(X_train,y_train):
    print('\n{} of kfold {}'.format(i,kf.n_splits))
    xtr,xvl = X_train.loc[train_index],X_train.loc[test_index]
    ytr,yvl = y_train[train_index],y_train[test_index]
    
    model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
         gamma=0.0, max_delta_step=0.0, min_child_weight=1.0,
         missing=None, n_jobs=-1, objective='multi:softprob', random_state=42, reg_alpha=0.0,
         reg_lambda=1.0, scale_pos_weight=1.0, tree_method='auto', num_class = 9, silent=False)
    model.fit(xtr, ytr)
    pred=model.predict(xvl)
    pred_prob = model.predict_proba(xvl) 
    print('accuracy_score',accuracy_score(yvl,pred))
    print('log loss is : ', log_loss(yvl, pred_prob))
    print('f1 score ', f1_score(yvl, pred, average='weighted'))
    print(confusion_matrix(yvl,pred))
    i+=1


1 of kfold 3
accuracy_score 0.8011976047904191
log loss is :  0.6621332448946234
f1 score  0.7786787266070815
[[ 38   0   0   4   2   0   0   4   0]
 [  0   0   0   0   0   1   1   1   0]
 [  1   0   7   9   1   0   9   5   0]
 [  0   0   0 163   0   2   7   7   0]
 [  1   0   0   2  87   1   2   2   0]
 [  0   0   0   2   1  26  11   9   0]
 [  0   0   6   8   3   4  56  12   0]
 [  2   0   1   6   2   3   8 292   0]
 [  0   0   0   0   1   0   0  25   0]]

2 of kfold 3


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy_score 0.8149038461538461
log loss is :  0.6343234068651957
f1 score  0.791669857109646
[[ 39   0   0   0   1   1   4   2   0]
 [  0   0   1   0   0   0   0   1   0]
 [  0   0   7   6   1   0   5  13   0]
 [  1   0   1 158   1   0  11   7   0]
 [  0   0   0   0  84   1   4   5   0]
 [  0   0   0   5   1  23   5  15   0]
 [  0   0   2   4   3   4  62  14   0]
 [  0   0   0   2   2   2   1 305   2]
 [  0   0   0   2   0   0   0  24   0]]

3 of kfold 3


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy_score 0.7980652962515115
log loss is :  0.6710893983547007
f1 score  0.7763822868177324
[[ 39   0   1   1   0   2   0   4   0]
 [  0   0   0   0   0   0   0   2   0]
 [  0   0   8   8   0   0   6   9   0]
 [  0   0   1 164   0   3   3   7   0]
 [  3   0   0   0  82   0   7   2   0]
 [  1   0   0   4   0  18  12  13   0]
 [  1   0   5  15   3   1  53  11   0]
 [  1   0   1   3   2   3   7 295   1]
 [  0   0   0   0   0   0   2  22   1]]


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Use grid search for best parameters

In [ ]:
xgbcl = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
         gamma=0.0, max_delta_step=0.0, min_child_weight=1.0,
         missing=None, n_jobs=-1, objective='multi:softprob', random_state=42, reg_alpha=0.0,
         reg_lambda=1.0, scale_pos_weight=1.0, tree_method='auto', num_class = 9, silent=False)

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)


# param_grid = { 
#     'colsample_bytree':[.75,1],
#     'learning_rate':[0.01,0.05,0.1,0.3,0.5],
#     'max_depth':[1,2,3,5,10, 20, 50, 100],
#     'subsample':[.75,1],
#     'n_estimators': list(range(50, 1000, 50))
# }

param_grid = { 
    'colsample_bytree':[.75,1],
    'learning_rate':[0.01,0.1,0.5],
    'max_depth':[1,50],
    'subsample':[.75,1],
    'n_estimators': list(range(50, 100, 50))
}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
grid_search = GridSearchCV(estimator=xgbcl, scoring=LogLoss, param_grid=param_grid, n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)

print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
grid_result.best_score_

In [ ]:
grid_result.best_params_

In [ ]:
#Set our final hyperparameters to the tuned values
xgbcl = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
         gamma=0.0, max_delta_step=0.0, min_child_weight=1.0,
         missing=None, n_jobs=-1, objective='multi:softprob', random_state=42, reg_alpha=0.0,
         reg_lambda=1.0, scale_pos_weight=1.0, tree_method='auto', num_class = 9, silent=False,
         colsample_bytree = grid_result.best_params_['colsample_bytree'], 
         learning_rate = grid_result.best_params_['learning_rate'], 
         max_depth = grid_result.best_params_['max_depth'], 
         subsample = grid_result.best_params_['subsample'], 
         n_estimators = grid_result.best_params_['n_estimators'])

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(estimator=xgbcl, X=X_train, y=y_train, cv=kfold, n_jobs=-1, 
                        scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'f1'])

print('Training 5-fold Cross Validation Results:\n')
print('AUC: ', scores['test_roc_auc'].mean())
print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean(), '\n')